# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 4 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[-1.73638668e-02,  0.00000000e+00, -8.11987277e-03, ...,
          -1.30902734e-02, -1.34414493e-03, -6.30739005e-03],
         [-1.93848714e-01, -2.12280467e-01, -2.69487441e-01, ...,
          -3.06648046e-01, -2.30226904e-01, -1.96743473e-01],
         [-2.68348396e-01, -2.86629409e-01, -3.42423439e-01, ...,
          -3.76332015e-01, -3.03701609e-01, -2.71256447e-01],
         ...,
         [-9.98975337e-01, -9.98841584e-01, -9.96024787e-01, ...,
          -9.55386639e-01, -9.84975278e-01, -9.91272390e-01],
         [-9.99578595e-01, -9.99459684e-01, -9.97734010e-01, ...,
          -9.75634158e-01, -9.92564321e-01, -9.96132433e-01],
         [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
          -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
 
        [[-1.73103306e-02,  0.00000000e+00, -8.10205285e-03, ...,
          -1.30698718e-02, -1.34197809e-03, -6.29818905e-03],
         [-1.93254337e-01, -2.11670950e-01, -2.68891245e-01, ...,
   

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(13725,)

array([ 7, 12,  4, 17,  3,  9, 14,  3,  0,  0, 10,  2, 12,  3,  1],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(13725, 128, 259)

array([[-0.07205169, -0.05472539, -0.05850169, ..., -0.06084616,
        -0.0555253 , -0.06296454],
       [-0.23574807, -0.25165832, -0.30234936, ..., -0.34071669,
        -0.27274629, -0.24320419],
       [-0.30578867, -0.32208526, -0.37220022, ..., -0.40863416,
        -0.34357452, -0.31456625],
       ...,
       [-0.99886495, -0.99888438, -0.99895239, ..., -0.99925578,
        -0.99919218, -0.9991554 ],
       [-0.9996506 , -0.9996624 , -0.99968928, ..., -0.99983454,
        -0.99981886, -0.99980623],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ]])

'mfcc'

(13725, 20, 259)

...

(13725, 84, 259)

array([[-22.15956306, -22.17663956, -22.30821991, ..., -22.349226  ,
        -22.2047081 , -22.18474007],
       [-22.48670578, -22.56513023, -22.71567535, ..., -22.64569855,
        -22.49061775, -22.40724182],
       [-22.87244987, -22.88685036, -23.0839138 , ..., -22.97397041,
        -22.74855804, -22.69841576],
       ...,
       [-42.51037598, -80.        , -80.        , ..., -80.        ,
        -80.        , -80.        ],
       [-42.75575256, -80.        , -80.        , ..., -80.        ,
        -80.        , -80.        ],
       [-42.94104004, -80.        , -80.        , ..., -80.        ,
        -80.        , -80.        ]])

'id'

(13725,)

'XC214099.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(4231,)

array([12, 18, 12,  1, 17, 16, 19, 16, 17,  8, 11,  4,  1,  9, 10],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(4231, 128, 259)

array([[-1.73596926e-02,  3.44094175e-08, -8.11852980e-03, ...,
        -1.30762281e-02, -1.34311256e-03, -6.31123409e-03],
       [-1.93800539e-01, -2.12224573e-01, -2.69393951e-01, ...,
        -3.06358129e-01, -2.30269626e-01, -1.96815208e-01],
       [-2.68296480e-01, -2.86559403e-01, -3.42290968e-01, ...,
        -3.75860423e-01, -3.03721368e-01, -2.71355629e-01],
       ...,
       [-9.98362124e-01, -9.97732460e-01, -9.94926810e-01, ...,
        -9.99149203e-01, -9.99104381e-01, -9.99068320e-01],
       [-9.99209702e-01, -9.99026000e-01, -9.98086751e-01, ...,
        -9.99572992e-01, -9.99560416e-01, -9.99544144e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

'mfcc'

(4231, 20, 259)

...

(4231, 84, 259)

array([[-2.26425767e-01, -2.54546463e-01, -3.99797261e-01, ...,
        -2.03850687e-01, -3.83407474e-02,  1.19209290e-07],
       [-4.55389321e-01, -5.32159269e-01, -6.78878844e-01, ...,
        -4.44713652e-01, -2.89330900e-01, -2.04249263e-01],
       [-7.69781888e-01, -7.86885083e-01, -9.90872085e-01, ...,
        -7.43024170e-01, -5.16102970e-01, -4.65163291e-01],
       ...,
       [-1.98852940e+01, -4.38798294e+01, -3.98226242e+01, ...,
        -5.25704956e+01, -5.40803909e+01, -4.84675674e+01],
       [-1.96396809e+01, -3.53625221e+01, -3.51839600e+01, ...,
        -5.14341125e+01, -5.04734726e+01, -5.58128815e+01],
       [-1.97824001e+01, -3.97351036e+01, -4.13765869e+01, ...,
        -5.98702164e+01, -5.15483704e+01, -6.97218628e+01]])

'id'

(4231,)

'XC383514.mp3'

# **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 63196)

In [18]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 63196)

Fit the model with training data

In [19]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [20]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2573859607657764


In [21]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.23      0.15       260
           1       0.45      0.51      0.48       424
           2       0.13      0.34      0.19       200
           3       0.18      0.31      0.23       318
           4       0.18      0.05      0.08        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.44      0.42      0.43       224
           8       0.13      0.54      0.21        50
           9       0.11      0.24      0.15        71
          10       0.22      0.11      0.15       146
          11       0.19      0.01      0.01       549
          12       0.38      0.30      0.34       415
          13       0.50      0.12      0.19       160
          14       0.03      0.07      0.04        46
          15       0.11      0.15      0.13       255
          16       0.92      0.93      0.92   

### **With MFCCs only**

In [22]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 5180)

In [23]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 5180)

In [24]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3056015126447648


In [26]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.30      0.68      0.42       260
           1       0.48      0.49      0.49       424
           2       0.20      0.61      0.30       200
           3       0.55      0.63      0.59       318
           4       0.00      0.00      0.00        73
           5       0.06      0.05      0.05        99
           6       0.00      0.00      0.00        86
           7       0.25      0.47      0.32       224
           8       0.12      0.12      0.12        50
           9       0.43      0.34      0.38        71
          10       0.14      0.03      0.05       146
          11       0.25      0.01      0.01       549
          12       0.49      0.43      0.46       415
          13       0.33      0.09      0.14       160
          14       0.02      0.04      0.03        46
          15       0.11      0.25      0.16       255
          16       0.50      0.37      0.43   

### **With Mel-Spectrogram Only**

In [27]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 33152)

In [28]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 33152)

In [29]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [30]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21933349090049634


In [31]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.28      0.19       260
           1       0.39      0.55      0.46       424
           2       0.17      0.35      0.23       200
           3       0.21      0.34      0.26       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.29      0.31      0.30       224
           8       0.14      0.54      0.22        50
           9       0.06      0.21      0.09        71
          10       0.33      0.16      0.22       146
          11       0.12      0.00      0.00       549
          12       0.30      0.18      0.22       415
          13       0.85      0.14      0.24       160
          14       0.01      0.02      0.01        46
          15       0.12      0.22      0.16       255
          16       0.47      0.32      0.38   

## **With Chroma Only**

In [32]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 3108)

In [33]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 3108)

In [34]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [35]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.18009926731269205


In [36]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.62      0.17       260
           1       0.25      0.36      0.29       424
           2       0.17      0.61      0.26       200
           3       0.59      0.15      0.24       318
           4       0.00      0.00      0.00        73
           5       1.00      0.01      0.02        99
           6       0.00      0.00      0.00        86
           7       0.14      0.33      0.20       224
           8       0.00      0.00      0.00        50
           9       0.38      0.04      0.08        71
          10       0.05      0.01      0.02       146
          11       0.00      0.00      0.00       549
          12       0.42      0.08      0.14       415
          13       0.00      0.00      0.00       160
          14       0.00      0.00      0.00        46
          15       0.40      0.47      0.43       255
          16       0.15      0.02      0.04   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.6649093907746979
F1 Score (Weighted): 0.138915698114714


## **With CQT Only**

In [37]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 21756)

In [38]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 21756)

In [39]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [40]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21058851335381706


In [41]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.06      0.12      0.08       260
           1       0.40      0.44      0.42       424
           2       0.08      0.26      0.12       200
           3       0.14      0.23      0.17       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.27      0.42      0.33       224
           8       0.35      0.36      0.35        50
           9       0.00      0.00      0.00        71
          10       0.06      0.02      0.03       146
          11       0.00      0.00      0.00       549
          12       0.30      0.31      0.31       415
          13       0.16      0.03      0.04       160
          14       0.00      0.00      0.00        46
          15       0.06      0.06      0.06       255
          16       0.93      0.91      0.92   

## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 38332)

In [43]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 38332)

In [44]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23398723705979674


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.30      0.20       260
           1       0.42      0.57      0.48       424
           2       0.14      0.34      0.20       200
           3       0.23      0.34      0.27       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.30      0.31      0.31       224
           8       0.13      0.54      0.21        50
           9       0.08      0.24      0.11        71
          10       0.30      0.16      0.21       146
          11       0.23      0.01      0.01       549
          12       0.38      0.27      0.31       415
          13       0.64      0.13      0.22       160
          14       0.03      0.07      0.04        46
          15       0.16      0.25      0.19       255
          16       0.46      0.31      0.37   

## **With MFCCs and Chroma**

In [47]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 8288)

In [48]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 8288)

In [49]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [50]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.30985582604585205


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.29      0.68      0.41       260
           1       0.50      0.51      0.51       424
           2       0.20      0.66      0.30       200
           3       0.55      0.63      0.59       318
           4       0.03      0.03      0.03        73
           5       0.04      0.03      0.04        99
           6       0.00      0.00      0.00        86
           7       0.26      0.49      0.34       224
           8       0.13      0.12      0.13        50
           9       0.32      0.32      0.32        71
          10       0.13      0.03      0.05       146
          11       0.07      0.00      0.00       549
          12       0.51      0.46      0.48       415
          13       0.39      0.09      0.15       160
          14       0.00      0.00      0.00        46
          15       0.12      0.23      0.15       255
          16       0.54      0.38      0.45   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7652253056947037
F1 Score (Weighted): 0.27678170382525463


## **With MFCCs and CQT**

In [52]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 26936)

In [53]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 26936)

In [54]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [55]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2505317891751359


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.10      0.21      0.14       260
           1       0.42      0.52      0.47       424
           2       0.08      0.35      0.13       200
           3       0.29      0.34      0.32       318
           4       0.06      0.03      0.04        73
           5       0.02      0.01      0.01        99
           6       0.00      0.00      0.00        86
           7       0.37      0.46      0.41       224
           8       0.18      0.22      0.20        50
           9       0.24      0.25      0.25        71
          10       0.23      0.10      0.14       146
          11       0.17      0.01      0.01       549
          12       0.29      0.29      0.29       415
          13       0.37      0.08      0.13       160
          14       0.04      0.13      0.06        46
          15       0.10      0.11      0.10       255
          16       0.94      0.93      0.93   

## **With Chroma and CQT**

In [57]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 24864)

In [58]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 24864)

In [59]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [60]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21247931930985584


In [61]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.06      0.13      0.08       260
           1       0.44      0.45      0.44       424
           2       0.08      0.29      0.13       200
           3       0.11      0.17      0.13       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.26      0.40      0.31       224
           8       0.31      0.30      0.30        50
           9       0.00      0.00      0.00        71
          10       0.06      0.01      0.02       146
          11       0.05      0.00      0.00       549
          12       0.30      0.33      0.32       415
          13       0.23      0.04      0.06       160
          14       0.01      0.02      0.01        46
          15       0.08      0.09      0.08       255
          16       0.92      0.92      0.92   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.6763190416626408
F1 Score (Weighted): 0.19666345023753873


## **With Mel-Spectrogram and Chroma**

In [62]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 36260)

In [63]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 36260)

In [64]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.21673363271094304


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.28      0.19       260
           1       0.37      0.50      0.43       424
           2       0.16      0.34      0.22       200
           3       0.21      0.31      0.25       318
           4       0.06      0.01      0.02        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.28      0.31      0.30       224
           8       0.13      0.52      0.21        50
           9       0.05      0.18      0.08        71
          10       0.33      0.18      0.23       146
          11       0.10      0.00      0.00       549
          12       0.30      0.20      0.24       415
          13       0.40      0.06      0.11       160
          14       0.02      0.04      0.03        46
          15       0.13      0.24      0.17       255
          16       0.50      0.37      0.42   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 54908)

In [68]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 54908)

In [69]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [70]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23871425194989365


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.08      0.17      0.11       260
           1       0.44      0.48      0.46       424
           2       0.15      0.36      0.21       200
           3       0.16      0.28      0.20       318
           4       0.00      0.00      0.00        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.43      0.41      0.42       224
           8       0.11      0.46      0.18        50
           9       0.08      0.17      0.11        71
          10       0.17      0.09      0.12       146
          11       0.07      0.00      0.00       549
          12       0.35      0.28      0.31       415
          13       0.48      0.07      0.12       160
          14       0.01      0.02      0.01        46
          15       0.11      0.16      0.13       255
          16       0.92      0.92      0.92   

## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 60088)

In [73]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 60088)

In [74]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [75]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.25478610257622314


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.11      0.21      0.14       260
           1       0.43      0.52      0.47       424
           2       0.14      0.34      0.20       200
           3       0.18      0.30      0.23       318
           4       0.14      0.04      0.06        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.45      0.41      0.43       224
           8       0.13      0.54      0.20        50
           9       0.10      0.21      0.13        71
          10       0.24      0.10      0.14       146
          11       0.25      0.01      0.01       549
          12       0.38      0.29      0.33       415
          13       0.48      0.09      0.15       160
          14       0.04      0.11      0.06        46
          15       0.11      0.16      0.13       255
          16       0.91      0.93      0.92   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(13725, 41440)

In [78]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(4231, 41440)

In [79]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [80]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2306783266367289


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.13      0.27      0.17       260
           1       0.41      0.57      0.48       424
           2       0.15      0.35      0.21       200
           3       0.24      0.36      0.29       318
           4       0.08      0.03      0.04        73
           5       0.00      0.00      0.00        99
           6       0.00      0.00      0.00        86
           7       0.32      0.32      0.32       224
           8       0.12      0.52      0.20        50
           9       0.07      0.24      0.11        71
          10       0.28      0.17      0.21       146
          11       0.25      0.01      0.01       549
          12       0.39      0.23      0.29       415
          13       0.60      0.07      0.13       160
          14       0.02      0.04      0.02        46
          15       0.13      0.20      0.16       255
          16       0.50      0.33      0.40   

## Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.257
3,chroma,1.0,0.180
8,chroma_cqt,1.0,0.212
4,cqt,1.0,0.211
2,melspectrogram,1.0,0.219
9,melspectrogram_chroma,1.0,0.217
12,melspectrogram_chroma_mfcc,1.0,0.231
10,melspectrogram_cqt,1.0,0.239
11,melspectrogram_cqt_mfcc,1.0,0.255
1,mfcc,1.0,0.306


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.24       0.73
3                    chroma      0.14       0.66
8                chroma_cqt      0.20       0.68
4                       cqt      0.19       0.67
2            melspectrogram      0.20       0.72
9     melspectrogram_chroma      0.20       0.71
10       melspectrogram_cqt      0.22       0.71
11  melspectrogram_cqt_mfcc      0.21       0.73
1                      mfcc      0.28       0.77
6               mfcc_chroma      0.28       0.77
7                  mfcc_cqt      0.23       0.73
5       mfcc_melspectrogram      0.22       0.73
